In [1]:
import argparse
import itertools
import json
import os
import random
import time
from functools import partial

import cv2
import imageio
import torchvision.transforms as T
import numpy as np
import torch
from decord import VideoReader, cpu
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms.functional import InterpolationMode
from tqdm import tqdm
import pandas as pd
# class ViedoMMEDataset(torch.utils.data.Dataset):

file_path = '/home/jovyan/shares/SR004.nfs2/lipengyi/Video-MME/videomme/test-00000-of-00001.parquet'

# 使用 pandas 读取 Parquet 文件
df = pd.read_parquet(file_path, engine='fastparquet')

# 显示 DataFrame 的内容
print(df.head())

print(df['question'][0])
print(df['options'][0])
print(df['answer'][0])
print(df['videoID'][0])
print(df['url'][0])
print(df['task_type'][0])

from datasets import load_dataset

ds = load_dataset("lmms-lab/Video-MME")
ds

test_dataset = ds['test']
test_dataset

print(test_dataset['question'][0])
print(test_dataset['url'][0])
print(test_dataset['videoID'][0])
print(test_dataset['task_type'][0])
print(test_dataset['options'][0])
print(test_dataset['answer'][0])

import os

video_data_1 = '/home/jovyan/shares/SR004.nfs2/lipengyi/Video-MME/data'
video_data_2 = '/home/jovyan/shares/SR004.nfs2/lipengyi/Video-MME/subtitle'

path = []
for file in os.listdir(video_data_1):
    if file.endswith(".srt"):
        full_path = os.path.join(video_data_1, file)
        path.append(full_path)
    if file.endswith(".mp4"):
        full_path = os.path.join(video_data_1, file)
        path.append(full_path)

for file in os.listdir(video_data_2):
    if file.endswith(".srt"):
        full_path = os.path.join(video_data_2, file)
        path.append(full_path)
    if file.endswith(".mp4"):
        full_path = os.path.join(video_data_2, file)
        path.append(full_path)
    
print(len(path))
# path

def find_video_path(video_id, path_list):
    # 遍历路径列表
    for path in path_list:
        if video_id in path:
            return path
    return None

list_data = []
for data in test_dataset:
    my_dict = {}
    # print(data)
    my_dict['videoID'] = data['videoID']
    my_dict['question'] = data['question']
    my_dict['duration'] = data['duration']
    my_dict['options'] = data['options']
    my_dict['answer'] = data['answer']
    my_dict['task_type'] = data['task_type']
    video_path = find_video_path(data['videoID'], path)
    my_dict['path'] = video_path
    list_data.append(my_dict)

  video_id duration     domain        sub_category  \
0      001    short  Knowledge  Humanity & History   
1      001    short  Knowledge  Humanity & History   
2      001    short  Knowledge  Humanity & History   
3      002    short  Knowledge  Humanity & History   
4      002    short  Knowledge  Humanity & History   

                                           url      videoID question_id  \
0  https://www.youtube.com/watch?v=fFjv93ACGo8  fFjv93ACGo8       001-1   
1  https://www.youtube.com/watch?v=fFjv93ACGo8  fFjv93ACGo8       001-2   
2  https://www.youtube.com/watch?v=fFjv93ACGo8  fFjv93ACGo8       001-3   
3  https://www.youtube.com/watch?v=N1cdUjctpG8  N1cdUjctpG8       002-1   
4  https://www.youtube.com/watch?v=N1cdUjctpG8  N1cdUjctpG8       002-2   

              task_type                                           question  \
0      Counting Problem  When demonstrating the Germany modern Christma...   
1  Information Synopsis                   What is the genre of this 

In [2]:
import argparse
import itertools
import json
import os
import random
import time
from functools import partial

import cv2
import imageio
import torchvision.transforms as T
import numpy as np
import torch
from decord import VideoReader, cpu
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms.functional import InterpolationMode
from tqdm import tqdm

# build_transform, dynamic_preprocess
def build_transform(is_train, input_size, pad2square=False, normalize_type='imagenet'):
    if normalize_type == 'imagenet':
        MEAN, STD = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
    elif normalize_type == 'clip':
        MEAN, STD = (0.4814546, 0.4578275, 0.40821073), (0.2686295, 0.2613025, 0.2757711)
    elif normalize_type == 'siglip':
        MEAN, STD = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
    else:
        raise NotImplementedError
    if is_train:  # use data augumentation
        transform = T.Compose([
            T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
            T.RandomChoice([T.Lambda(jpeg_degrade_functions[quality]) for quality in qualities]),
            T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
            T.ToTensor(),
            T.Normalize(mean=MEAN, std=STD)
        ])
    else:
        if pad2square is False:  # now we use this transform function by default
            transform = T.Compose([
                T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
                T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
                T.ToTensor(),
                T.Normalize(mean=MEAN, std=STD)
            ])
        else:
            transform = T.Compose([
                T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
                T.Lambda(lambda img: expand2square(img, tuple(int(x * 255) for x in MEAN))),
                T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
                T.ToTensor(),
                T.Normalize(mean=MEAN, std=STD)
            ])

    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    # print(f'width: {width}, height: {height}, best_ratio: {best_ratio}')
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=6, image_size=448, use_thumbnail=False):
    
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

In [3]:
class VideoMMEDataset(torch.utils.data.Dataset):
    def __init__(self, list_data, prompt, question_prompt, num_segments=16, input_size=224,
                 dynamic_image_size=False, use_thumbnail=False, max_num=6):
        self.data_list = list_data

        self.prompt = prompt
        self.question_prompt = question_prompt
        self.input_size = input_size
        self.num_segments = num_segments
        self.dynamic_image_size = dynamic_image_size
        self.use_thumbnail = use_thumbnail
        self.max_num = max_num
        #self.transform = build_transform(is_train=False, input_size=input_size)

        self.decord_method = {
            'video': self.read_video,
        }

    def __len__(self):
        return len(self.data_list)

    def get_index(self, bound, fps, max_frame, first_idx=0):
        if bound:
            start, end = bound[0], bound[1]
        else:
            start, end = -100000, 100000
        start_idx = max(first_idx, round(start * fps))
        end_idx = min(round(end * fps), max_frame)
        seg_size = float(end_idx - start_idx) / self.num_segments
        frame_indices = np.array([
            int(start_idx + (seg_size / 2) + np.round(seg_size * idx))
            for idx in range(self.num_segments)
        ])
        return frame_indices

    
    # 读取视频，根据对应的 video_path
    def read_video(self, video_path, bound=None):
        vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
        max_frame = len(vr) - 1
        fps = float(vr.get_avg_fps())

        images_group = list()
        frame_indices = self.get_index(bound, fps, max_frame, first_idx=0)
        for frame_index in frame_indices:
            img = Image.fromarray(vr[frame_index].asnumpy())
            images_group.append(img)

        return images_group

    # question 模版
    def qa_template(self, data):
        question = data['question']#f"Question: {data['question']}\n"
        question += 'Options:\n'
        answer = data['answer']
        answer_idx = -1
        for idx, c in enumerate(data['options']):
            question += f"{c}\n"
            if c == answer:
                answer_idx = idx
                
        question = question.rstrip()
        return question, answer

    # 制作数据模版
    def __getitem__(self, idx):
        
        data_item = self.data_list[idx]
        video_path = data_item['path']
        duration = data_item['duration']
        # print(duration)
        
        if not os.path.exists(video_path):
            print(f"Warning: Video file not found at {video_path}, skipping this item.")
            return None  # 或者返回一个特定的占位符数据结构

    
        #special_tokens = '\n'.join(['Frame{}: <image>'.format(i + 1) for i in range(len(image_list))])
        question, answer = self.qa_template(data_item)
        #question = special_tokens + '\n' + self.prompt + '\n' + question + self.question_prompt
        question = question + self.question_prompt
    
    
    
        return {
            'question': question,
            'answer': answer,
            'path':video_path,
            'task_type': data_item['task_type'],
            'duration': duration
        }
    

In [4]:
prompt = 'Carefully watch the video and pay attention to the cause and sequence of events, the detail and movement of objects, and the action and pose of persons. Based on your observations, select the best option that accurately addresses the question.\n'
question_prompt = '\nOnly give the best option.'

dataset = VideoMMEDataset(
    list_data=list_data,        
    prompt=prompt,               
    question_prompt=question_prompt,
    num_segments=16,              
    input_size=448,              
    dynamic_image_size=True,     
    use_thumbnail=False,          
    max_num=6                     
)

In [5]:
dataset[0]

{'question': 'When demonstrating the Germany modern Christmas tree is initially decorated with apples, candles and berries, which kind of the decoration has the largest number?Options:\nA. Apples.\nB. Candles.\nC. Berries.\nD. The three kinds are of the same number.\nOnly give the best option.',
 'answer': 'C',
 'path': '/home/jovyan/shares/SR004.nfs2/lipengyi/Video-MME/data/fFjv93ACGo8.mp4',
 'task_type': 'Counting Problem',
 'duration': 'short'}

In [6]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from models import build_model

Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'


In [7]:
model_path = "/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft"#"/home/jinyang06/models/VideoLaVIT-v1/language_model_sft"
model_dtype='bf16'

max_video_clips = 16
device_id = 0
torch.cuda.set_device(device_id)
device = torch.device('cuda')

seed = 42
#torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# For Multi-Modal Understanding
runner = build_model(model_path=model_path, model_dtype=model_dtype, understanding=True, 
        device_id=device_id, use_xformers=False, max_video_clips=max_video_clips,)

Loading Video LaVIT Model Weight from /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft, model precision: bf16
Not used {}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft were not used when initializing VideoLaVITLlamaForCausalLM: ['model.motion_tokenizer.quantize.cluster_size', 'model.motion_tokenizer.quantize.embedding.embed_avg', 'model.motion_tokenizer.quantize.embedding.cluster_size', 'model.motion_tokenizer.quantize.embedding.initted']
- This IS expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The Visual Vocab Size is 16384
The llama tokenizer vocab size is 32000
The maximal clip number is 16


In [8]:
from transformers import AutoModel, AutoTokenizer, AutoProcessor

video_path = '/home/jovyan/shares/SR004.nfs2/lipengyi/Video-MME/data/fFjv93ACGo8.mp4'
prompt = "When demonstrating the Germany modern Christmas tree is initially decorated with apples, candles and berries, which kind of the decoration has the largest number?\nOptions:\nA. Apples.\nB. Candles.\nC. Berries.\nD. The three kinds are of the same number.\nOnly give the best option."
answer = "C"
output = runner({"video": video_path, "text_input": prompt}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
print(output)

/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be

The Germany modern Christmas tree is initially decorated with apples, candles and berries. A total of 6 apples are used for decoration, 24 candles are used for lighting and 48 decorative berries are used. So, the option A is the best option.


In [9]:
prompt = "When demonstrating the Germany modern Christmas tree is initially decorated with apples, candles and berries, which kind of the decoration has the largest number?"
answer = "C"
output = runner({"video": video_path, "text_input": prompt}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
print(output)

Apples, candles, and berries are all present in equal amounts on the Germany modern Christmas tree.


In [10]:
import transformers

tokenizer = transformers.LlamaTokenizer.from_pretrained(model_path, use_fast=False, padding_side='left')
tokenizer.pad_token = tokenizer.unk_token

In [11]:
import torch
from torch.utils.data import DataLoader
from functools import partial


def collate_fn(batches, tokenizer):
    
    questions = [_['question'] for _ in batches]
    video_path = [_['path'] for _ in batches]
    answer = [_['answer'] for _ in batches]
    task_types = [_['task_type'] for _ in batches]
    
    return questions, video_path, answer, task_types

dataloader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=1,
        num_workers=1,
        pin_memory=True,
        drop_last=False,
        collate_fn=partial(collate_fn, tokenizer=tokenizer)
    )

iterator = iter(dataloader)
first_batch = next(iterator)


question = first_batch[0]
video_path = first_batch[1]
answer = first_batch[2]
task_type = first_batch[3]

print(question)
print(answer)
print(task_type)
print(video_path)

['When demonstrating the Germany modern Christmas tree is initially decorated with apples, candles and berries, which kind of the decoration has the largest number?Options:\nA. Apples.\nB. Candles.\nC. Berries.\nD. The three kinds are of the same number.\nOnly give the best option.']
['C']
['Counting Problem']
['/home/jovyan/shares/SR004.nfs2/lipengyi/Video-MME/data/fFjv93ACGo8.mp4']


In [ ]:
y_pred = []
y_real = []

progress_bar = tqdm(
        dataloader, total=len(dataloader)
    )

for step, batch in enumerate(progress_bar, start=1):
    question = batch[0]
    video_path = batch[1]
    answer = batch[2]
    task_type = batch[3]

    # print(pixel_values.size())
    # print(question)
    # print(answer)
    # print(num_patches_list)
    # print(task_type)
    
    # with autocast():
        # 执行推理
    outputs = runner({"video": video_path, "text_input": question}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]

    y_pred.append(outputs.strip())
    y_real.append(answer[0].strip())
    if ((step-1)%100 == 0):
        print("="*20 + "output" + "="*20)
        print(outputs)
        print("="*20 + "real answers" + "="*20)
        print(answer[0], flush = True)

  0%|          | 0/2700 [00:00<?, ?it/s]

====================output====================
A. Candles.
====================real answers====================
C


  0%|          | 3/2700 [00:54<13:22:58, 17.86s/it]

In [2]:
y_pred[:10]

NameError: name 'y_pred' is not defined

In [10]:
y_preds = [i.split('.')[0] for i in y_pred]

result = [1 if i == j else 0 for i, j in zip(y_real, y_preds)]

print("VideoLAVIT-MME, accuracy is: " + str(sum(result) / len(result)))

InternVL-1B-16-frames-MME, accuracy is: 0.4014814814814815
